**FEEL FREE TO UPVOTE**  （＾ｖ＾）

In [ ]:

#v20提交成绩得到了3.688， cv 成绩3.6519
#使用authorized_flag 进行特征分别处理,收到了奇效
#使用countVectorizer 对类别变量进行编码，内存存在压力

In [1]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
import time
warnings.filterwarnings('ignore')
np.random.seed(4950)

In [5]:
df_train=pd.read_csv('df_train_2_extra.csv')
df_test=pd.read_csv('df_test_2_extra.csv')

In [6]:
exclude_features = []
exclude_features += ['card_id', 'first_active_month','target','outliers']
df_train_columns = [c for c in df_train.columns if c not in exclude_features ]
target = df_train['target']
# del df_train['target']

In [7]:
len(df_train_columns)
len(exclude_features)

4

In [28]:
df_train2 = df_train.copy()
df_test2=df_test.copy()
outlier = df_train2.outliers
#删除异常值编码，下面是没有异常值的训练集
#下面这个特征对于识别异常点有一定帮助，但是训练model_without_outliers的时候需要删除这几个特征。
for f in ['feature_1','feature_2','feature_3','month','dayofweek']:
    del df_train2[f + '_outliers_mean']
    del df_train2[f + '_outliers_sum']
    del df_test2[f + '_outliers_mean']
    del df_test2[f + '_outliers_sum']
exclude_features = []
exclude_features += ['card_id', 'first_active_month','target','outliers']
df_train_columns2 = [c for c in df_train2.columns if c not in exclude_features ]
trn_data = lgb.Dataset(df_train[df_train_columns], label=target)#, categorical_feature=categorical_feats)

## Above is the same with wang, I call it wang's features

### I replace wang's stratified CV with normal lgb.cv

In [40]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 42,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 6,
         "random_state": 4950}

In [13]:
trn_data = lgb.Dataset(df_train[df_train_columns], label=target)#, categorical_feature=categorical_feats)

In [14]:
#trn_data = lgb.Dataset(df_train[df_train_columns], label=target, categorical_feature=['month','dayofweek','weekofyear'])

In [15]:
lgb_cv = lgb.cv(param, trn_data, 10000, early_stopping_rounds=600, verbose_eval=200, stratified=False)

[200]	cv_agg's rmse: 3.68175 + 0.0289848
[400]	cv_agg's rmse: 3.66228 + 0.0272734
[600]	cv_agg's rmse: 3.65542 + 0.0265762
[800]	cv_agg's rmse: 3.65252 + 0.0261455
[1000]	cv_agg's rmse: 3.65146 + 0.0260665
[1200]	cv_agg's rmse: 3.6511 + 0.0258999
[1400]	cv_agg's rmse: 3.65081 + 0.0255834
[1600]	cv_agg's rmse: 3.65098 + 0.0253746
[1800]	cv_agg's rmse: 3.6508 + 0.0252724
[2000]	cv_agg's rmse: 3.65098 + 0.0252186
[2200]	cv_agg's rmse: 3.65135 + 0.0249002


In [38]:
trn_data2 = lgb.Dataset(df_train2[df_train_columns2], label=target)#, categorical_feature=categorical_feats)

In [41]:
lgb_cv2 = lgb.cv(param, trn_data2, 10000, early_stopping_rounds=600, verbose_eval=200, stratified=False)

[200]	cv_agg's rmse: 3.68112 + 0.0293679
[400]	cv_agg's rmse: 3.66178 + 0.0282259
[600]	cv_agg's rmse: 3.65495 + 0.0276025
[800]	cv_agg's rmse: 3.65234 + 0.0272248
[1000]	cv_agg's rmse: 3.65133 + 0.0271494
[1200]	cv_agg's rmse: 3.65085 + 0.0271195
[1400]	cv_agg's rmse: 3.65061 + 0.026787
[1600]	cv_agg's rmse: 3.65039 + 0.0265507
[1800]	cv_agg's rmse: 3.65051 + 0.0262945
[2000]	cv_agg's rmse: 3.65087 + 0.0263697
[2200]	cv_agg's rmse: 3.65096 + 0.0265505


In [44]:
len(lgb_cv2['rmse-mean'])

1634

In [42]:
clf2 = lgb.train(param, trn_data2, len(lgb_cv2['rmse-mean']), valid_sets=(trn_data2), verbose_eval=200)

[200]	training's rmse: 3.57441
[400]	training's rmse: 3.48674
[600]	training's rmse: 3.43079
[800]	training's rmse: 3.38841
[1000]	training's rmse: 3.35138
[1200]	training's rmse: 3.31784
[1400]	training's rmse: 3.28627
[1600]	training's rmse: 3.25528


In [43]:
y_pred = clf2.predict(df_test2[df_train_columns2], num_iteration=clf2.best_iteration)

pd.DataFrame({
        'card_id': df_test.card_id,
        'target': y_pred
}).to_csv('wang_lgb_cv_pur_date_1_updated4.csv', index=False)
#LB:3.678

In [45]:
clf3 = lgb.train(param, trn_data2, 2000, valid_sets=(trn_data2), verbose_eval=200) # let's overtrain

[200]	training's rmse: 3.57441
[400]	training's rmse: 3.48674
[600]	training's rmse: 3.43079
[800]	training's rmse: 3.38841
[1000]	training's rmse: 3.35138
[1200]	training's rmse: 3.31784
[1400]	training's rmse: 3.28627
[1600]	training's rmse: 3.25528
[1800]	training's rmse: 3.22687
[2000]	training's rmse: 3.19972


In [46]:
y_pred = clf3.predict(df_test2[df_train_columns2], num_iteration=clf3.best_iteration)

pd.DataFrame({
        'card_id': df_test.card_id,
        'target': y_pred
}).to_csv('wang_lgb_cv_pur_date_1_updated5.csv', index=False)
#LB: 3.677

In [ ]:
lgb_cv

Results without categorical_feature=['month','dayofweek','weekofyear'].
[200]	cv_agg's rmse: 3.69075 + 0.0319068
[400]	cv_agg's rmse: 3.6695 + 0.0304884
[600]	cv_agg's rmse: 3.66173 + 0.0294372
[800]	cv_agg's rmse: 3.65849 + 0.0290465
[1000]	cv_agg's rmse: 3.65738 + 0.0285758
[1200]	cv_agg's rmse: 3.65658 + 0.0280419
[1400]	cv_agg's rmse: 3.65644 + 0.0274495
[1600]	cv_agg's rmse: 3.65649 + 0.0271829
[1800]	cv_agg's rmse: 3.65663 + 0.0268914

In [16]:
lgb_cv['rmse-mean'].index(min(lgb_cv['rmse-mean']))

1743

In [21]:
min(lgb_cv['rmse-mean'])

3.6507604085335066

In [22]:
len(lgb_cv['rmse-mean'])

1744

In [23]:
lgb_cv.keys()

dict_keys(['rmse-mean', 'rmse-stdv'])

Original wang results:

[200]	cv_agg's rmse: 3.68161 + 0.0299797
[400]	cv_agg's rmse: 3.66225 + 0.0286057
[600]	cv_agg's rmse: 3.65503 + 0.0277111
[800]	cv_agg's rmse: 3.65259 + 0.0276703
[1000]	cv_agg's rmse: 3.65156 + 0.027532
[1200]	cv_agg's rmse: 3.65109 + 0.0273339
[1400]	cv_agg's rmse: 3.65112 + 0.0268877
[1600]	cv_agg's rmse: 3.65121 + 0.0265736
[1800]	cv_agg's rmse: 3.65127 + 0.0266131

lgb.cv show's that num_boost_round is around 2000, so we train a model with num_boost_round=2000

In [20]:
clf = lgb.train(param, trn_data, len(lgb_cv['rmse-mean']), valid_sets=(trn_data), verbose_eval=200)

[200]	training's rmse: 3.57365
[400]	training's rmse: 3.48585
[600]	training's rmse: 3.42889
[800]	training's rmse: 3.38562
[1000]	training's rmse: 3.34979
[1200]	training's rmse: 3.31615
[1400]	training's rmse: 3.28493
[1600]	training's rmse: 3.25431


Results without categorical_feature=['month','dayofweek','weekofyear'].

[200]	training's rmse: 3.58917
[400]	training's rmse: 3.49676
[600]	training's rmse: 3.43963
[800]	training's rmse: 3.39575
[1000]	training's rmse: 3.35879
[1200]	training's rmse: 3.32572
[1400]	training's rmse: 3.29513
[1600]	training's rmse: 3.26594
[1800]	training's rmse: 3.23792
[2000]	training's rmse: 3.21083

Original Wang's results:
[200]	training's rmse: 3.57275
[400]	training's rmse: 3.48472
[600]	training's rmse: 3.42865
[800]	training's rmse: 3.3851
[1000]	training's rmse: 3.34906
[1200]	training's rmse: 3.3155
[1400]	training's rmse: 3.28252
[1600]	training's rmse: 3.25342
[1800]	training's rmse: 3.22531
[2000]	training's rmse: 3.19685

predict and save

In [24]:
y_pred = clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration)

pd.DataFrame({
        'card_id': df_test.card_id,
        'target': y_pred
}).to_csv('wang_lgb_cv_pur_date_1_updated3.csv', index=False)
#LB: 3.678

## Outlier detection using wang's features

the param is same as regression and just adjust to binary classification

best num_boost_round is 500, so we use 500 to train a classifer

In [26]:
df_train_columns

['feature_3',
 'auth_flag_purchase_amount_sum',
 'auth_flag_authorized_flag_sum',
 'auth_flag_authorized_flag_mean',
 'auth_flag_authorized_flag_std',
 'auth_flag_card_id_size',
 'auth_encoder_category_2_auth_mean_mean',
 'auth_encoder_category_2_auth_sum_sum',
 'auth_encoder_category_3_auth_mean_mean',
 'auth_encoder_category_3_auth_sum_sum',
 'auth_encoder_state_id_auth_mean_mean',
 'auth_encoder_state_id_auth_sum_sum',
 'auth_encoder_subsector_id_auth_mean_mean',
 'auth_encoder_subsector_id_auth_sum_sum',
 'auth_encoder_merchant_category_id_auth_mean_mean',
 'auth_encoder_merchant_category_id_auth_sum_sum',
 'auth_encoder_city_id_auth_mean_mean',
 'auth_encoder_city_id_auth_sum_sum',
 'agg_per_month_totalmonth_lag_mean',
 'agg_per_month_totalmonth_lag_std',
 'agg_per_month_totalagg_per_month_totalauthorized_flag_sum_mean',
 'agg_per_month_totalagg_per_month_totalauthorized_flag_sum_std',
 'agg_per_month_totalagg_per_month_totalauthorized_flag_mean_mean',
 'agg_per_month_totalagg_per

In [28]:
df_train2 = df_train.copy()
df_test2=df_test.copy()
outlier = df_train2.outliers
#删除异常值编码，下面是没有异常值的训练集
#下面这个特征对于识别异常点有一定帮助，但是训练model_without_outliers的时候需要删除这几个特征。
for f in ['feature_1','feature_2','feature_3','month','dayofweek']:
    del df_train2[f + '_outliers_mean']
    del df_train2[f + '_outliers_sum']
    del df_test2[f + '_outliers_mean']
    del df_test2[f + '_outliers_sum']
exclude_features = []
exclude_features += ['card_id', 'first_active_month','target','outliers']
df_train_columns2 = [c for c in df_train2.columns if c not in exclude_features ]
trn_data = lgb.Dataset(df_train[df_train_columns], label=target)#, categorical_feature=categorical_feats)

In [29]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 42,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 6,
         "scale_pos_weight": 15,
         "random_state": 4950}

In [30]:
trn_data = lgb.Dataset(df_train2[df_train_columns2], label=outlier)#, categorical_feature=categorical_feats)

In [31]:
lgb_cv = lgb.cv(param, trn_data, 10000, early_stopping_rounds=600, verbose_eval=200, stratified=False)

[200]	cv_agg's auc: 0.904452 + 0.0048605
[400]	cv_agg's auc: 0.905061 + 0.00456461
[600]	cv_agg's auc: 0.904718 + 0.00484739
[800]	cv_agg's auc: 0.904136 + 0.00520001


In [32]:
lgb_cv['auc-mean'][-1]

0.9051801367503455

In [33]:
len(lgb_cv['auc-mean'])

333

In [34]:
clf = lgb.train(param, trn_data, len(lgb_cv['auc-mean']), verbose_eval=100, valid_sets=(trn_data))

[100]	training's auc: 0.954415
[200]	training's auc: 0.96213
[300]	training's auc: 0.967929


In [35]:
te_prob = clf.predict(df_test2[df_train_columns2])

In [36]:
te_prob.min(), te_prob.max()

(0.0010467757547024475, 0.8333526258519501)

In [37]:
df_outliers=pd.DataFrame()
df_outliers['card_id']=df_test['card_id']
df_outliers['probability']=te_prob
df_outliers.to_csv('wang_lgb_cv_pur_date_1_updated_outliers_pred_uodated3.csv',index=False)

In [219]:
wang_out=pd.read_csv('wang_lgb_cv_pur_date_1_updated')
wang_out['outlier_prob1'] = te_prob

In [221]:
_40=wang_out.sort_values('outlier_prob1',ascending=False)

In [222]:
top20 = _40.card_id.values[:20].tolist()

In [223]:
wang_out.loc[wang_out.card_id.isin(top20), 'target'] = -33.21928095
wang_out[['card_id', 'target']].to_csv('wang_lgb_cv_pur_date_1_updated_top20.csv', index=False)

In [224]:
top20_50 = _40.card_id.values[20:50].tolist()

In [225]:
top20_50_df = wang_out[wang_out.card_id.isin(top20_50)]

In [226]:
top20_50_target = top20_50_df.outlier_prob1 * (-33.21928095)

In [227]:
wang_out.loc[wang_out.card_id.isin(top20_50), 'target'] = top20_50_target

In [230]:
wang_out[['card_id', 'target']].to_csv('wang_lgb_cv_pur_date_1_updated_top20_top20-50.csv', index=False)

### Now let's try to merge results with the model without outliers.
### First, predict without outliers

In [243]:
df_train['target']=target

In [246]:
df_train2=df_train[df_train['outliers']==0]

In [247]:
df_train2.shape

(199710, 257)

In [248]:
target2=df_train2.target

In [92]:
len(df_train_columns),len(df_train_columns2)

(253, 243)

In [237]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 42,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 48,
         "random_state": 4950}

In [249]:
trn_data2 = lgb.Dataset(df_train2[df_train_columns], label=target2)#, categorical_feature=categorical_feats)

In [250]:
lgb_cv = lgb.cv(param, trn_data2, 10000, early_stopping_rounds=600, verbose_eval=200, stratified=False)

[200]	cv_agg's rmse: 1.58191 + 0.00883915
[400]	cv_agg's rmse: 1.56523 + 0.00798252
[600]	cv_agg's rmse: 1.55947 + 0.0075402
[800]	cv_agg's rmse: 1.5571 + 0.00732745
[1000]	cv_agg's rmse: 1.55593 + 0.0070912
[1200]	cv_agg's rmse: 1.55515 + 0.00691628
[1400]	cv_agg's rmse: 1.55474 + 0.00682905
[1600]	cv_agg's rmse: 1.55452 + 0.00677001
[1800]	cv_agg's rmse: 1.55431 + 0.00669755
[2000]	cv_agg's rmse: 1.55412 + 0.00663288
[2200]	cv_agg's rmse: 1.55398 + 0.00654708
[2400]	cv_agg's rmse: 1.55386 + 0.0064701
[2600]	cv_agg's rmse: 1.55382 + 0.00645527
[2800]	cv_agg's rmse: 1.55381 + 0.00636882
[3000]	cv_agg's rmse: 1.55379 + 0.00623851
[3200]	cv_agg's rmse: 1.55373 + 0.00625909
[3400]	cv_agg's rmse: 1.55379 + 0.00623813
[3600]	cv_agg's rmse: 1.55377 + 0.00623746


In [63]:
threshold10=pd.read_csv('threshold_feature_elimination10_8extra_test10_without_outliers.csv',squeeze=False)

In [73]:
for i in range(len(threshold10)):
    if 'feature' in threshold10.loc[i].values[0]:
        print(threshold10.loc[i].values[0])

feature_1_outliers_mean
feature_1_outliers_sum
feature_2_outliers_mean
feature_2_outliers_sum
feature_3_outliers_sum
feature_1_1
feature_1_3
feature_1_4
feature_2_2
feature_2_3


In [84]:
df_train['feature_3_outliers_mean'].unique()

array([0.01142752, 0.01028283])

In [85]:
df_train['feature_2_3'].unique()

array([0, 1], dtype=int64)

In [86]:
threshold10.loc[0].values[0]

'auth_encoder_category_3_auth_mean_mean'

In [87]:
clf = lgb.train(param, trn_data2, 3600, valid_sets=(trn_data2), verbose_eval=200)

[200]	training's rmse: 3.57441
[400]	training's rmse: 3.48674
[600]	training's rmse: 3.43079


KeyboardInterrupt: 

In [256]:
y_pred = clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration)

pd.DataFrame({
        'card_id': df_test.card_id,
        'target': y_pred
}).to_csv('wang_lgb_cv_pur_date_1_updated_without_outliers', index=False)

### Let's combine models with and without outliers

In [88]:
model_with_outliers=pd.read_csv('wang_lgb_cv_pur_date_1_updated')
model_without_outliers=pd.read_csv('wang_lgb_cv_pur_date_1_updated_without_outliers')


In [89]:
probability_df=model_with_outliers.copy()
probability_df['prob']=te_prob
del probability_df['target']

In [90]:
probability_df.sort_values('prob',ascending=False).head(18000).prob.min()

0.1481680365769966

In [91]:
probability_df.sort_values('prob',ascending=False).head(18000).prob.max()

0.8333526258519501

In [282]:
#Let's use 18000 as threshould value as we have used before
index_high_prob=probability_df.sort_values('prob',ascending=False).head(18000).index

In [83]:
index_high_prob2=probability_df.sort_values('prob',ascending=False).head(25000).index

NameError: name 'probability_df' is not defined

In [276]:
index_high_prob

Int64Index([ 96354,  88754,  78078, 115682, 109960,   5008, 109705, 118268,
              5707,  57626,
            ...
            116990,  53485,  88674,  88770,  52048, 115876,  51772,  26502,
             11497,  66954],
           dtype='int64', length=18000)

In [292]:
mixed_model=model_without_outliers.copy()
mixed_model.loc[index_high_prob,'target']=model_with_outliers.loc[index_high_prob,'target']
mixed_model.to_csv('wang_lgb_cv_pur_date_1_updated_mixed.csv',index=False)

### Now let's use previous methods to change top 20 and top 20-50 results

In [297]:
mixed_model.loc[mixed_model.card_id.isin(top20),'target']=-33.21928095

In [301]:
mixed_model.loc[mixed_model.card_id.isin(top20_50),'target']=top20_50_target

In [305]:
mixed_model.to_csv('wang_lgb_cv_pur_date_1_updated_mixed_top20_top20_50.csv',index=False)

##LB score: 3.663